In [1]:
# This script should take the em_centroids h5_file and query the agglo_id
#TODO: create reference lm stack 
# TODO: add paths to sample
# TODO: change neuron for centroid in lut
# TODO: Read image information from json file
# TODO: Create functions to get info from h5 files
# Standard library imports
import os
import time
from tqdm import tqdm
import logging


# Third-party imports
import numpy as np
import pandas as pd
import h5py
import tifffile
import napari
import matplotlib.pyplot as plt
from scipy.interpolate import RBFInterpolator
import dill
from skimage.measure import regionprops, label
from google.cloud import bigquery
from google.auth import exceptions

# Custom imports
from brainmaps_api_fcn.equivalence_requests import EquivalenceRequests
from brainmaps_api_fcn.subvolume_requests import SubvolumeRequest
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack

<Figure size 640x480 with 0 Axes>

In [2]:
def get_agglo_group_with_retry(sa, volume_id, stack_change, centroid_xyz, max_retries=5):
    """Get agglomeration group with retry mechanism."""
    for attempt in range(max_retries):
        try:
            sr = SubvolumeRequest(sa, volume_id)
            vol = sr.get_subvolume(centroid_xyz, size=[1,1,1], change_stack_id=stack_change)
            agglo_id = int(np.unique(vol[vol>0])[0])

            er = EquivalenceRequests(sa, volume_id, stack_change)
            return er.get_groups(agglo_id)
        except exceptions.RefreshError:
            if attempt == max_retries - 1:
                raise
            print(f"Try {attempt+1}/{max_retries}")
            time.sleep(attempt)


def get_neuron_segments(lut_path, neuron_id):
    """Get all segment IDs for a given neuron"""
    with h5py.File(lut_path, 'r') as f:
        neuron_path = f'neurons/neuron_{neuron_id}'
        if neuron_path not in f:
            return []

        # Get all segment IDs from the segments group
        segments = f[neuron_path]['agglo_segments'][:]
        # Convert segment IDs from strings to integers


        return segments


def get_segments_by_agglo_id(lut_path, agglo_id):
    """Get all segment IDs for a given agglomeration ID"""
    segments = []
    with h5py.File(lut_path, 'r') as f:
        # Iterate through all neurons to find matching agglo_id
        for neuron_name, neuron_group in f['neurons'].items():
            if 'agglo_id' in neuron_group.attrs:
                if neuron_group.attrs['agglo_id'] == agglo_id:
                    # Get segments from this neuron
                    if 'agglo_segments' in neuron_group:
                        segments = neuron_group['agglo_segments'][:]
                        break
    return segments


def find_neurons_in_mask(lut_path, mask_path):
    """Find neurons with centroids inside a 3d mask"""
    # Open the Paintera zarr mask
    mask = tifffile.imread(mask_path, mode='r')
    neurons_inside = []

    with h5py.File(lut_path, 'r') as f:
        for neuron_name, neuron_group in f['neurons'].items():
            # Get the neuroglancer coordinates of the centroid
            # We use ng coordinates since Paintera mask is in the same space
            centroid = neuron_group['em_centroid_ng'][:] // 16

            #print(centroid)

            # Round coordinates to integers for indexing
            x, y, z = np.round(centroid).astype(int)

            # Check if the centroid is within mask bounds
            if (0 <= x < mask.shape[2] and
                    0 <= y < mask.shape[1] and
                    0 <= z < mask.shape[0]):

                # Check if the point is inside the mask (non-zero value)
                if mask[z, y, x] > 0:
                    neuron_id = int(neuron_name.split('_')[1])
                    agglo_id = neuron_group.attrs.get('agglo_id', None)
                    neurons_inside.append({
                        'neuron_id': neuron_id,
                        'agglo_id': agglo_id,
                        'centroid_zyx': centroid[::-1]
                    })

    return neurons_inside


def get_neurons_with_attribute(lut_path, attribute_name, attribute_value, operator="=="):
    """
    Get neurons where attribute matches the comparison with threshold

    Args:
        lut_path: Path to HDF5 file
        attribute_name: Name of attribute to check
        operator: String specifying comparison ('>', '<', '>=', '<=', '==')
        threshold: Value to compare against
    """
    neuron_with_attribute = []
    operators = {
        '>': lambda x, y: x > y,
        '<': lambda x, y: x < y,
        '>=': lambda x, y: x >= y,
        '<=': lambda x, y: x <= y,
        '==': lambda x, y: x == y
    }

    if operator not in operators:
        raise ValueError(f"Operator must be one of {list(operators.keys())}")

    with h5py.File(lut_path, 'r') as f:
        for neuron, neuron_group in f['neurons'].items():
            if attribute_name in neuron_group.attrs:
                stored_value = neuron_group.attrs[attribute_name]
                if operators[operator](stored_value, attribute_value) and "agglo_id" in neuron_group.attrs:
                    # print(stored_value)
                    neuron_with_attribute.append(neuron_group.attrs["agglo_id"])
                    #print(f"Neuron {neuron}: {attribute_name} = {stored_value}")

    return neuron_with_attribute

def extract_dataset(h5_path, group_path):
    """Extract datasets and attributes from HDF5 group into a dictionary"""
    extracted_data = {}
    
    with h5py.File(h5_path, 'r') as f:
        group = f[group_path]
        
        # Get datasets
        for key, val in group.items():
            extracted_data[key] = val[()]
        
        # Get attributes
        for key, val in group.attrs.items():
            extracted_data[key] = val
            
    return extracted_data 

def apply_transformation(points, transform):
    # Convert to homogeneous coordinates
    homogeneous_points = np.column_stack((points, np.ones(len(points))))
    # Apply transformation
    transformed_points = homogeneous_points @ transform.T
    # Return to 3D coordinates
    return transformed_points[:, :3]


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    z, y, x = np.round(point).astype(int)
    if 0 <= z < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= x < mask.shape[2]:
        return mask[z, y, x]
    return False





In [3]:
client = bigquery.Client(project="fmi-friedrich")

# Step 1: Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
# Step 2: Load experiment configuration
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)


# Input: volume data
sa = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\Reconstruction\fmi-friedrich-ebb67584e50d.json"
volume_id = r"280984173682:montano_rm2_ngff:raw_230701_seg_240316fb"
stack_change = "240705d_rsg9_spl"

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('neuron_processing.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [4]:
em_centroids_path = os.path.join(exp.paths.em_path, f"{exp.sample.id}_em_centroids.h5")


In [5]:
# Read centroids from HDF5 file and do transformation to neuroglancer
with h5py.File(em_centroids_path, 'r') as f:
    # Get centroids coordinates
    data = f['centroids/coordinates'][:]
    centroids = data[:, 1:]

    # Get transformation parameters
    transformation_matrix = f['metadata/transformations/bigwarp2neuroglancer/transformation_matrix'][:]
    rotated_cropped_stack_center_shift = f[
                                             'metadata/transformations/bigwarp2neuroglancer/rotated_cropped_stack_center_shift'][
                                         :]
    rotated_stack_center_shift = f['metadata/transformations/bigwarp2neuroglancer/rotated_stack_center_shift'][:]
    cropped_shift = f['metadata/transformations/bigwarp2neuroglancer/cropped_shift'][:]
    ng_shift = f['metadata/transformations/bigwarp2neuroglancer/ng_shift'][:]
    downsampled_factor = f['metadata/transformations/bigwarp2neuroglancer/downsampled_factor'][:]
    zyx2xyz = f['metadata/transformations/bigwarp2neuroglancer'].attrs['zyx2xyz']
    # Apply transformations to get Neuroglancer coordinates
    # 1. Center points around origin
    centered_points = centroids - rotated_cropped_stack_center_shift

    # 2. Apply transformation matrix
    transformed_centered = np.dot(centered_points, transformation_matrix.T)

    # 3. Move to target space and apply shifts
    transformed_points = (transformed_centered +
                          rotated_stack_center_shift +
                          cropped_shift)
    if f['metadata/transformations/bigwarp2neuroglancer/downsampled_factor']:
        downsampled_factor = f['metadata/transformations/bigwarp2neuroglancer/downsampled_factor'][:]
        transformed_points = transformed_points * downsampled_factor

    # 4. Convert from ZYX to XYZ if needed
    if f['metadata/transformations/bigwarp2neuroglancer'].attrs['zyx2xyz']:
        transformed_points = transformed_points[:, ::-1]

    # 5. Correct for shift in neuroglancer
    transformed_points = transformed_points + ng_shift


In [6]:
transformation_path = os.path.join(exp.paths.clem_path, f"{exp.sample.id}_transformations.h5")

In [69]:
with h5py.File(transformation_path, 'w') as hdf5_file:
    images_group = hdf5_file.create_group('images_info')
    
    # EM stack in neuroglancer
    em_stack_ng_group = images_group.create_group('em_stack_ng')
    em_stack_ng_group.attrs['path'] = volume_id
    em_stack_ng_group.attrs['mask_path'] = []
    em_stack_ng_group.attrs['voxel_size'] = [.01, 0.01, 0.25] 
    em_stack_ng_group.attrs['voxel_unit'] = 'um'
    em_stack_ng_group.attrs['dim_order'] = 'xyz'
    
    # EM stack in bigwarp space
    em_stack_bw_group = images_group.create_group('em_stack_bw')
    em_stack_bw_group.attrs['path'] = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_fine_aligned_downsampled_16_em_stack_cropped_woResin_rough_rotated_to_LM.tif')
    em_stack_bw_group.attrs['mask_path'] = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_fine_aligned_downsampled_16_em_stack_cropped_woResin_rough_rotated_to_LM_cp_mask_filtered.tif')
    em_stack_bw_group.attrs['voxel_size'] = [.16, 0.16, 0.4] 
    em_stack_bw_group.attrs['voxel_unit'] = 'um'
    em_stack_bw_group.attrs['dim_order'] = 'zyx'
    
    # EM stack downsampled
    em_stack_ds_group = images_group.create_group('em_stack_ds')
    em_stack_ds_group.attrs['path'] = os.path.join(exp.paths.em_path,'fine_aligned_downsampled_4_em_stack_cropped.tif')
    em_stack_ds_group.attrs['mask_path'] = []
    em_stack_ds_group.attrs['voxel_size'] = [.16, 0.16, 0.4]  # 16x downsampled
    em_stack_ds_group.attrs['voxel_unit'] = 'um'
    em_stack_ds_group.attrs['dim_order'] = 'zyx'
    
    # LM stack raw
    lm_stack_raw_group = images_group.create_group('lm_stack_raw')
    lm_stack_raw_group.attrs['path'] = os.path.join(exp.paths.anatomy_path, 'raw', '20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif')
    lm_stack_raw_group.attrs['mask_path'] = []
    lm_stack_raw_group.attrs['voxel_size'] = [1.0, 0.4, 0.4]  # [1.0, 0.4, 0.4] in µm
    lm_stack_raw_group.attrs['voxel_unit'] = 'um'
    lm_stack_raw_group.attrs['dim_order'] = 'zyx'
    
    # LM stack in bigwarp space
    lm_stack_bw_group = images_group.create_group('lm_stack_bw')
    lm_stack_bw_group.attrs['path'] = os.path.join(exp.paths.anatomy_path, 'processed', 'flipped_upsampled_clahe_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif')
    lm_stack_bw_group.attrs['mask_path'] = os.path.join(exp.paths.clem_path, 'em_stack_warped_fov_lm_res_lm_cp_masks.tif') 
    lm_stack_bw_group.attrs['voxel_size'] = [0.4, 0.4, 0.4]  # [0.4, 0.4, 0.4] in µm
    lm_stack_bw_group.attrs['voxel_unit'] = 'um'
    lm_stack_bw_group.attrs['dim_order'] = 'zyx'
    
    # LM stack reference
    lm_stack_ref_group = images_group.create_group('lm_stack_ref')
    lm_stack_ref_group.attrs['path'] = os.path.join(exp.paths.anatomy_path, 'aligned_to_reference')
    lm_stack_ref_group.attrs['mask_path'] = []
    lm_stack_ref_group.attrs['voxel_size'] = [1.0, 0.4, 0.4]  # Same as bigwarp space
    lm_stack_ref_group.attrs['voxel_unit'] = 'um'
    lm_stack_ref_group.attrs['dim_order'] = 'zyx'
    
    # LM planes 
    lm_planes_group = images_group.create_group('lm_planes')
    lm_planes_group.attrs['path'] = os.path.join(exp.paths.trials_path, 'processed', f'sum_elastic_corrected_trials_{exp.sample.id}.tif')
    lm_planes_group.attrs['mask_path'] = os.path.join(exp.paths.trials_path, 'masks', f'masks_{exp.sample.id}_params_cp_-1-ft_0.4-st_0.01-resample_True_augment=False.tif')
    lm_planes_group.attrs['voxel_size'] = [0.4, 0.4] 
    lm_planes_group.attrs['voxel_unit'] = 'um'
    lm_planes_group.attrs['dim_order'] = 'xy'

    #### TRANSFOMRATIONS ####
    transform_group = hdf5_file.create_group('transformations')
    
    # EM space transformations
    em_group = transform_group.create_group('em_transformations')
    ng2bw_group = em_group.create_group('ng2bw')  # neuroglancer to bigwarp
    ng2ds_group = em_group.create_group('ng2ds')  # bigwarp to downsampled
    
        # ng2bw transformations
    ng2bw_group.create_dataset('transformation_matrix', data=transformation_matrix)
    ng2bw_group.create_dataset('rotated_cropped_stack_center_shift', data=rotated_cropped_stack_center_shift)
    ng2bw_group.create_dataset('rotated_stack_center_shift', data=rotated_stack_center_shift)
    ng2bw_group.create_dataset('cropped_shift', data=cropped_shift)
    ng2bw_group.create_dataset('ng_shift', data=ng_shift)
    ng2bw_group.attrs['downsampled_factor'] = downsampled_factor
    ng2bw_group.attrs['zyx2xyz'] = zyx2xyz
    
        # ng2ds transformations
    ng2ds_group.create_dataset('ng_shift', data=ng_shift)
    ng2ds_group.create_dataset('cropped_shift', data=cropped_shift)
    ng2ds_group.attrs['downsampled_factor'] = downsampled_factor
    
    # LM space transformations
    lm_group = transform_group.create_group('lm_transformations')
    raw2bw_group = lm_group.create_group('raw2bw')  # bigwarp to raw
    raw2ref = lm_group.create_group('raw2ref')  # raw to reference
    plane2bw = lm_group.create_group('plane2bw')  # raw to reference

        # raw2bw transformations
    raw2bw_group.attrs['flip_axis'] = 1  # Vertical flip (y-axis)
    raw2bw_group.attrs['upsample_z'] = 2.5  # Factor to get from 1.0 to 0.4 in z
    
        # plane2stack transformations
    doubling_factor = 2 if exp.params_lm.doubling else 1
    plane2bw.attrs['transformation_matrices'] = [os.path.join(exp.paths.trials_path, 'processed', f'registration_tform_lm_plane0{plane_nr}_lm_stack_bw.npy') for plane_nr in range(exp.params_lm.n_planes*doubling_factor)]

    # CLEM transformation (with 
    clem_group = transform_group.create_group('clem_transformations')

        # em2lm transformations
    clem_group.attrs['rbf_interpolator_path'] = os.path.join(exp.paths.clem_path,
            f'{exp.sample.id}_em2lm_interpolator.dill')
    


In [7]:
lut_path = os.path.join(exp.paths.clem_path, f"{exp.sample.id}_lut.h5")

In [11]:
# Finding agglomeration IDs fom centroids

#with h5py.File(lut_path, 'w') as hdf5_file:
    error_neurons = []
    error_neurons_coordinates = []
    # Create metadata and store transformations
    metadata_group = hdf5_file.create_group('metadata')

    metadata_group.attrs['sample_id'] = exp.sample.id
    metadata_group.attrs['volume_id'] = volume_id
    metadata_group.attrs['stack_change'] = stack_change

    neurons_group = hdf5_file.create_group('neurons')

    # Process each neuron
    for neuron_id in tqdm(range(len(centroids)), desc="Processing neurons"):
        try:
            neuron_name = f'neuron_{neuron_id}'
            if neuron_name not in neurons_group:
                neuron_group = neurons_group.create_group(f'neuron_{neuron_id}')
                # Adding EM_CENTROID_BW and EM_CENTROID_NG
                neuron_group.create_dataset('em_centroid_bw', data=centroids[neuron_id])
                neuron_group.create_dataset('em_centroid_ng', data=transformed_points[neuron_id])
                # error_neurons.append((neuron_id, "Neuron not found in file"))
                # continue

            neuron_group = neurons_group[neuron_name]
            ng_centroid = transformed_points[neuron_id]
            print(f"Processing neuron {neuron_id} at position {ng_centroid}")

            # Get agglomeration ID and segments
            centroid_xyz = np.round(ng_centroid).astype(int)
            sr = SubvolumeRequest(sa, volume_id)
            vol = sr.get_subvolume(centroid_xyz, size=[30,30,30], change_stack_id=stack_change)

            if not np.any(vol > 0):
                error_neurons.append((neuron_id, "No agglomeration ID found at position"))
                error_neurons_coordinates.append(ng_centroid)
                continue

            unique_ids, counts = np.unique(vol[vol > 0], return_counts=True)
            total_voxels = np.sum(counts)

            # Sort IDs by count
            sort_idx = np.argsort(counts)[::-1]
            unique_ids = unique_ids[sort_idx]
            counts = counts[sort_idx]

            if counts[0] / total_voxels > 0.9:
                agglo_id = int(unique_ids[0])
                er = EquivalenceRequests(sa, volume_id, stack_change)
                agglo_group = er.get_groups(agglo_id)

                for agglo_id, segment_ids in agglo_group.items():
                    neuron_group.attrs['agglo_id'] = agglo_id
                    if 'agglo_segments' in neuron_group:
                        del neuron_group['agglo_segments']
                    neuron_group.create_dataset('agglo_segments', data=segment_ids)
                    print(f"Assigned dominant agglo_id {agglo_id} ({counts[0]/total_voxels*100:.1f}% of volume)")
            else:
                segments_length_list = []
                for id, count in zip(unique_ids, counts):
                    er = EquivalenceRequests(sa, volume_id, stack_change)
                    agglo_group = er.get_groups(int(id))

                    for aid, segments in agglo_group.items():
                        print(f"Agglo id {aid}: {count/total_voxels*100:.1f}% ({len(segments)} segments)")
                        if len(segments) > 1:
                            segments_length_list.append((aid, segments))

                if len(segments_length_list) == 1:
                    agglo_id, segment_ids = segments_length_list[0]
                    neuron_group.attrs['agglo_id'] = agglo_id
                    if 'agglo_segments' in neuron_group:
                        del neuron_group['agglo_segments']
                    neuron_group.create_dataset('agglo_segments', data=segment_ids)
                    print(f"Assigned agglo_id {agglo_id} based on segment count")
                else:
                    error_neurons.append((neuron_id, f"No dominant ID found ({len(unique_ids)} IDs present)"))
                    error_neurons_coordinates.append(ng_centroid)

        except Exception as e:
            error_neurons.append((neuron_id, str(e)))
            error_neurons_coordinates.append(ng_centroid)
            print(f"Error processing neuron {neuron_id}: {e}")

print(f"Neurons with errors: {len(error_neurons)}")


Processing neurons:   0%|          | 0/10 [00:00<?, ?it/s]

Processing neuron 0 at position [7731.34535966 9385.17245292   79.51508963]


Processing neurons:  10%|█         | 1/10 [00:04<00:43,  4.81s/it]

Assigned dominant agglo_id 16922938 (100.0% of volume)
Processing neuron 1 at position [8065.5852694  9082.63058164   98.29708354]


Processing neurons:  20%|██        | 2/10 [00:09<00:40,  5.02s/it]

...

Assigned dominant agglo_id 19389875 (100.0% of volume)
Processing neuron 8 at position [ 8022.64278768 10586.27844408   125.15429498]


Processing neurons:  90%|█████████ | 9/10 [00:43<00:05,  5.12s/it]

Assigned dominant agglo_id 19421304 (100.0% of volume)
Processing neuron 9 at position [7553.27841031 8727.93276047  111.04081633]


Processing neurons: 100%|██████████| 10/10 [00:48<00:00,  4.84s/it]

Assigned dominant agglo_id 16074434 (100.0% of volume)
Neurons with errors: 0


In [8]:
# Get segments for neuron 0
segments = get_neuron_segments(lut_path, 0)
print(f"Segments for neuron 0: {segments[:10]}")

# Get segments for agglomeration ID 12345
segments = get_segments_by_agglo_id(lut_path, 55233441)
print(f"Segments for agglo_id {55233441}: {segments[:10]}")

Segments for neuron 0: [16922938 16922939 16922941 16923026 16923028 16923041 16923063 17740001
 17755708 17755723]
Segments for agglo_id 55233441: [55233441 55234191 56065381 56065778 56065816 56081333 56081776 56081807
 56081910 56899353]


# Adding em_centroids_ds

In [126]:
# Extracting transformations needed
ng2ds_t = extract_dataset(transformation_path, 'transformations/em_transformations/ng2ds')            
print(ng2ds_t)

# Convert to ds and add to lut
with h5py.File(lut_path, 'r+') as f:

    for neuron_name, neuron_group in tqdm(f['neurons'].items()):
        # Get centroid from this neuron
        centroid = neuron_group['em_centroid_ng'][:]
        centroid_transformed = centroid[::-1] // ng2ds_t['downsampled_factor'] - ng2ds_t['cropped_shift']
        # Store EM centroid downsampled (ds)
        if 'em_centroid_ds' in neuron_group:
            del neuron_group['em_centroid_ds']
        neuron_group.create_dataset('em_centroid_ds', data=centroid_transformed)



{'cropped_shift': array([  0, 175,  73]), 'ng_shift': array([1, 1, 1]), 'downsampled_factor': array([16, 16, 16])}


100%|██████████| 9462/9462 [00:10<00:00, 879.34it/s] 


# Adding lm_centroids_bw

In [ ]:
# Load an existing interpolator
with h5py.File(transformation_path, 'r') as f:
    rbf_interpolator_path = f['transformations/clem_transformations'].attrs['rbf_interpolator_path']
    
with open(rbf_interpolator_path, 'rb') as f:
    rbf_interpolator = dill.load(f)

# Warping em centroids to lm centroids individually
with h5py.File(lut_path, 'r+') as f:
    for neuron, neuron_group in tqdm(f['neurons'].items(), desc="Warping em centroids to lm centroids"):
        if 'em_centroid_bw' in neuron_group:
            # Get EM centroid
            em_centroid = neuron_group['em_centroid_bw'][:].reshape(1, -1)
            
            # Transform to LM space
            lm_centroid = rbf_interpolator(em_centroid).flatten()
            
            # Store LM centroid
            if 'lm_centroid_bw' in neuron_group:
                del neuron_group['lm_centroid_bw']
            neuron_group.create_dataset('lm_centroid_bw', data=lm_centroid)


# Adding lm_planes_ids

In [70]:
# plane_nr = 6
# t_form_new = np.load(r"D:\montruth\PycharmProjects\samplePy\notebooks\registration_tform_lm_plane06_lm_stack_nopad.npy")

# Load transformation matrices
plane2bw_t_paths = extract_dataset(transformation_path, 'transformations/lm_transformations/plane2bw')['transformation_matrices']
print(plane2bw_t_paths)
t_forms = []
for path in plane2bw_t_paths:
    t_form = np.load(path)
    t_forms.append(t_form)
t_forms = np.array(t_forms)    

# Load plane images 
lm_planes_info = extract_dataset(transformation_path, 'images_info/lm_planes')

lm_planes = tifffile.imread(lm_planes_info['path'])[:, -1] # taking only last ones
lm_planes_flipped = np.array([np.flip(plane, axis=-1) for plane in lm_planes])

# Load plane masks (from cellpose)

lm_planes_mask = tifffile.imread(lm_planes_info['mask_path'])[:, -1]
lm_planes_mask_flipped = np.array([np.flip(mask, axis=-1) for mask in lm_planes_mask])


# Load lm stack bw (used for BigWarp)
lm_stack_bw_info = extract_dataset(transformation_path, 'images_info/lm_stack_bw')

lm_stack_bw = tifffile.imread(lm_stack_bw_info['path'])[:,0]
lm_stack_mask_bw = tifffile.imread(lm_stack_bw_info['mask_path'])
print(lm_stack_mask_bw.shape)

# Load lm stack bw
lm_stack_raw_info = extract_dataset(transformation_path, 'images_info/lm_stack_raw')

lm_stack_raw = load_tiff_as_hyperstack(lm_stack_raw_info['path'], n_channels= 2)[0]

# 
# # Flip LM stack vertically (along y-axis)
# lm_stack_bw_flipped = np.flip(lm_stack_bw, axis=2)



['\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\trials\\processed\\registration_tform_lm_plane00_lm_stack_bw.npy'
 '\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\trials\\processed\\registration_tform_lm_plane01_lm_stack_bw.npy'
 '\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\trials\\processed\\registration_tform_lm_plane02_lm_stack_bw.npy'
 '\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\trials\\processed\\registration_tform_lm_plane03_lm_stack_bw.npy'
 '\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\trials\\processed\\registration_tform_lm_plane04_lm_stack_bw.npy'
 '\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\trials\\processed\\registration_tform_lm_plane05_lm_stack_bw.npy'
 '\\\\tungsten-nas.fmi.ch\\tungsten\\scr

In [124]:
viewer = napari.Viewer()

# Add LM stack in bigwarp space
viewer.add_image(
    lm_stack_bw,
    name=f'LM stack bw',
    blending='additive'
)

# Add mask
viewer.add_image(
    lm_stack_mask_bw,
    name=f'LM Stack Mask',
    blending = "additive"

)

# Add LM plane
viewer.add_image(
    lm_planes_flipped,
    name=f'LM Planes Flipped',
    blending='additive'
)

# Add mask
viewer.add_image(
    lm_planes_mask_flipped,
    name=f'LM Plane Masks Flipped',
    blending='additive'
)


<Image layer 'LM Plane Masks Flipped' at 0x1deece4c310>

In [73]:

# Get centroids from all planes
all_centroids_3d = []
for plane_nr in range(lm_planes_mask_flipped.shape[0]):
    # Get centroids for current plane
    plane_props = regionprops(label(lm_planes_mask_flipped[plane_nr]))
    plane_centroids = np.array([prop.centroid for prop in plane_props])
    
    # Convert to 3D coordinates (z,y,x)
    plane_centroids_3d = np.column_stack((
        np.zeros(len(plane_centroids)),  # z coordinate
        plane_centroids[:, 0],           # y coordinate
        plane_centroids[:, 1]            # x coordinate
    ))
    
    # Transform points using corresponding transformation matrix
    t_form = t_forms[plane_nr]
    transformed_centroids = apply_transformation(plane_centroids_3d, t_form)
    
    # Add to viewer with different color per plane
    viewer.add_points(
        transformed_centroids, 
        name=f'Transformed Centroids Plane {plane_nr}',
        #face_color=plt.cm.tab10(plane_nr % 10),  # Cycle through 10 colors
        size=10
    )
    
    all_centroids_3d.append(transformed_centroids)

# Combine all transformed centroids into single array if needed
all_transformed_centroids = np.vstack(all_centroids_3d)


In [192]:
from napari.utils.transforms import Affine
# Loop through all planes and their transformations
for plane_nr in range(lm_planes_flipped.shape[0]):
    # Get current plane and transform
    lm_plane = lm_planes_flipped[plane_nr]
    t_form = t_forms[plane_nr]
    
    # Convert to napari's Affine format
    napari_affine = Affine(affine_matrix=t_form)
    
    # Convert 2D plane to 3D (add z-dimension)
    lm_plane_3d = lm_plane[np.newaxis, :, :]  # Shape becomes (1, H, W)
    
    # Add to viewer with different color per plane
    viewer.add_image(
        lm_plane_3d,
        name=f'Tilted Plane {plane_nr}',
        affine=napari_affine,
        blending='additive',
        colormap=plt.cm.viridis(plane_nr/lm_planes_flipped.shape[0]),  # Different color per plane
        opacity=0.5
    )


D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requeste

In [120]:
for plane_nr in range(lm_planes_mask_flipped.shape[0]):
    # Get current plane and transform
    lm_plane = lm_planes_mask_flipped[plane_nr]
    t_form = t_forms[plane_nr]
    
    # Convert to napari's Affine format
    napari_affine = Affine(affine_matrix=t_form)
    
    # Convert 2D plane to 3D (add z-dimension)
    lm_plane_3d = lm_plane[np.newaxis, :, :]  # Shape becomes (1, H, W)
    
    # Add to viewer with different color per plane
    viewer.add_image(
        lm_plane_3d,
        name=f'Tilted Plane {plane_nr}',
        affine=napari_affine,
        blending='additive',
        colormap=plt.cm.viridis(plane_nr/lm_planes_mask_flipped.shape[0]),  # Different color per plane
        opacity=0.5
    )


D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\layers\utils\_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requeste

In [140]:
# Assigning lm_stack_bw_labels

mapping_lm_bw_em_ng = []
with h5py.File(lut_path, "r+") as f:
    
    for neuron, neuron_info in f["neurons"].items():
        if "lm_centroid_bw" in neuron_info:
            lm_centroid_bw_ii = f["neurons"][neuron]["lm_centroid_bw"][:]
            print(lm_centroid_bw_ii)
            mask_value = is_within_mask(lm_centroid_bw_ii,lm_stack_mask_bw)
            print(mask_value)
            f["neurons"][neuron].attrs["lm_stack_bw_mask_label"] = mask_value
            mapping_lm_bw_em_ng.append((neuron, mask_value))
            

df_mapping = pd.DataFrame(mapping_lm_bw_em_ng, columns=["neuron_id", "lm_stack_bw_mask_label"])

[116.81865841 348.35750448 282.45937319]
3425
[108.89449801 347.81311578 269.19612843]
3197
[ 91.33075593 340.92220185 288.5119571 ]
2707
[123.35946899 335.23803581 262.69897475]
3679
[103.19270963 287.68557384 172.55417798]
2996
[ 87.68328488 283.66883593 174.25061586]
2537
[ 45.31529077 273.76545658 172.07650173]
1223
[140.91140357 290.40611983 198.59907691]
4315
[ 55.72370309 275.74639916 171.04946768]
1600
[ 74.07559996 274.69730862 200.35735812]
2218
[255.56953329 307.18986893 257.79775364]
0
[ 86.72623371 272.39155117 229.52125921]
2631
[202.89324132 281.61502406 264.52226587]
0
[134.55017929 279.31711602 250.88293964]
4030
[ 52.78494029 320.58421254 259.21317764]
0
[216.16331615 288.4622946  272.97263258]
0
[157.97086433 281.01628235 267.28794174]
4609
[ -2.14999221 245.38655293 227.13680071]
False
[219.81715734 286.30003961 289.82156691]
0
[ 69.86544973 264.95115091 268.0770567 ]
1959
[ 89.2581715  263.58237195 278.63153432]
2659
[ 99.74033223 264.99895269 291.25687902]
2992
[2

In [155]:
df_mapping.head()

,neuron_id,lm_stack_bw_mask_label
0,neuron_0,3425
1,neuron_1,3197
2,neuron_10,2707
3,neuron_100,3679
4,neuron_1000,2996


In [173]:
with h5py.File(lut_path, "r+") as f:
    
    # Process each plane
    for plane_nr in range(lm_planes_mask_flipped.shape[0]):
        # Get centroids for current plane
        plane_props = regionprops(label(lm_planes_mask_flipped[plane_nr]))
        plane_centroids = np.array([prop.centroid for prop in plane_props])
        
        # Convert to 3D coordinates
        plane_centroids_3d = np.column_stack((
            np.zeros(len(plane_centroids)),
            plane_centroids[:, 0],
            plane_centroids[:, 1]
        ))
        
        # Transform centroids
        transformed_centroids = apply_transformation(plane_centroids_3d, t_forms[plane_nr])
        
        # Store mapping between original and transformed centroids
        for i, (orig_centroid, trans_centroid) in enumerate(zip(plane_centroids, transformed_centroids)):
            mask_value = is_within_mask(trans_centroid, lm_stack_mask_bw)
            if mask_value and mask_value > 0:    
                print(mask_value)
                neuron_id = df_mapping[df_mapping["lm_stack_bw_mask_label"]==mask_value]["neuron_id"].iloc[0] if not df_mapping[df_mapping["lm_stack_bw_mask_label"]==mask_value].empty else None
                print(neuron_id)
                if neuron_id is None:
                    continue
                if neuron_id in f["neurons"]:
                    # Store original plane centroid with plane number
                    if 'lm_centroid_plane_flip' in f["neurons"][neuron_id]:
                        del f["neurons"][neuron_id]['lm_centroid_plane_flip']
                    f["neurons"][neuron_id].create_dataset('lm_centroid_plane_flip', 
                                              data=[plane_nr, *orig_centroid])
                    
                    # Store transformed centroid
                    if 'lm_centroid_plane_bw' in f["neurons"][neuron_id]:
                        del f["neurons"][neuron_id]['lm_centroid_plane_bw']
                    f["neurons"][neuron_id].create_dataset('lm_centroid_plane_bw', 
                                              data=trans_centroid)

                print(f["neurons"][neuron_id]["lm_centroid_bw"][:])
                print(f["neurons"][neuron_id]["em_centroid_ng"][:])
                print(f["neurons"][neuron_id]["lm_centroid_plane_flip"][:])
                print(f["neurons"][neuron_id]["lm_centroid_plane_bw"][:])


5975
neuron_6775
[215.4060855  140.57776457 274.53108922]
[ 5654.48215796 11452.98444106  3311.40892948]
[  0.           6.67391304 271.8115942 ]
[212.00811456 134.13302412 272.50220555]
5903
neuron_7572
[208.45340299 136.72546194 171.51725845]
[6858.46078859 7888.78815655 3630.48348611]
[  0.          14.27150538 162.03763441]
[212.79511434 141.77448249 167.45304813]
6035
neuron_6859
[217.66434974 155.88723411 207.37069768]
[6248.30277306 8938.90152801 3328.88907753]
[  0.          26.94701987 208.02649007]
[212.54898597 153.75293475 211.5163929 ]
5979
neuron_6800
[211.25204446 163.71709333 156.90966906]
[6781.88383589 7132.15966387 3295.0280112 ]
[  0.          35.70588235 150.42647059]
[212.98708767 162.33057744 156.41176855]
5998
neuron_6133
[216.67390894 164.55938907 252.48846566]
[ 5817.1832833  10370.26149471  3069.72618107]
[  0.          37.50877193 248.99122807]
[212.32590416 163.72497505 250.76349721]
5862
neuron_6220
[210.02598321 170.28753249 208.04690866]
[6415.28580392 8

In [191]:
test_centroids = {}
neuron_id = 691
with h5py.File(lut_path, "r") as f:
    test_centroids["lm_centroid_bw"] = f["neurons"][f"neuron_{neuron_id}"]["lm_centroid_bw"][:]
    test_centroids["lm_centroid_plane_bw"] = f["neurons"][f"neuron_{neuron_id}"]["lm_centroid_plane_bw"][:]
    
# viewer.add_image(lm_stack_bw)
# viewer.add_image(lm_stack_mask_bw)
viewer.add_points(test_centroids["lm_centroid_bw"], name = "lm_centroid_bw_i", size= 5, face_color= "cyan")
viewer.add_points(test_centroids["lm_centroid_plane_bw"], name = "lm_centroid_plane_bw_i", size = 5, face_color = "orange")
z, y, x = test_centroids["lm_centroid_bw"]
viewer.camera.center = [z, y, x]


# Adding attribute in_OB

In [31]:
# Loading ob mask (e.g. created with paintera)
ob_mask_path = os.path.join(exp.paths.em_path, 'masks', 'fine_aligned_downsampled_4_em_stack_cropped_ob_mask.tif')
ob_mask = tifffile.imread(ob_mask_path)
    
ng2bw_t = extract_dataset(transformation_path, 'transformations/em_transformations/ng2bw')            


In [33]:
viewer.add_image(np.flip(ob_mask))

<Image layer 'Image' at 0x1de62163a90>

In [127]:
# Create attribute 'in_OB' depending of mask
with h5py.File(lut_path, 'r+') as f:

    for neuron_name, neuron_group in tqdm(f['neurons'].items()):
        # Get centroid from this neuron

        centroid = neuron_group['em_centroid_ng'][:]
        if ng2bw_t['zyx2xyz'] == True:
            centroid = centroid[::-1]
        centroid_transformed = centroid // ng2bw_t['downsampled_factor'] - ng2bw_t['cropped_shift']

        mask_value = ob_mask[tuple(centroid_transformed.astype(int))]

        if mask_value == 1:
            neuron_group.attrs['in_OB'] = True
        elif mask_value == 0:
            neuron_group.attrs['in_OB'] = False

100%|██████████| 9462/9462 [00:07<00:00, 1192.06it/s]


In [39]:
# Extract centroids outside and inside of OB
neurons_in_ob = []
centroids_in_ob = []
centroids_outside_ob = []
with h5py.File(lut_path, 'r') as f:
    for neuron_name, neuron_group in tqdm(f['neurons'].items()):
        # First check if in_OB exists and is True
        if neuron_group.attrs.get("agglo_id", False):
            if neuron_group.attrs["in_OB"] == True:
                neurons_in_ob.append(neuron_group.attrs['agglo_id'])
                centroids_in_ob.append(neuron_group['lm_centroid_bw'][:])
            else:
                centroids_outside_ob.append(neuron_group['lm_centroid_bw'][:])

100%|██████████| 9462/9462 [00:04<00:00, 2163.76it/s]


In [129]:
# Visualize
viewer = napari.Viewer()
viewer.add_labels(ob_mask)
viewer.add_points(centroids_in_ob, face_color='green')
viewer.add_points(centroids_outside_ob, face_color='red')

2025-02-05 15:45:48,458 - WARNING - DirectWrite: CreateFontFaceFromHDC() failed (Indicates an error in an input file such as a font file.) for QFontDef(Family="", pointsize=12, pixelsize=16, styleHint=5, weight=50, stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0, lfHeight=-16) dpi=96
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Points layer 'Points [1]' at 0x17423f46950>

# Adding attribute g_coeff, r_coeff

In [140]:
# Assign IN information

mask_colored_stack_c0_path = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\masks\mask_manderscoeff_c0_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif"
mask_colored_stack_c1_path = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\masks\mask_manderscoeff_c1_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif"

mask_colored_stack_c0 = tifffile.imread(mask_colored_stack_c0_path)
mask_colored_stack_c1 = tifffile.imread(mask_colored_stack_c1_path)

# outside_lm_points = []

# Get mask dimensions for bounds checking
z_max, y_max, x_max = mask_colored_stack_c0.shape

with h5py.File(lut_path, 'r+') as f:
    for neuron, neuron_group in f['neurons'].items():
        if 'lm_centroid_bw' in neuron_group:
            # Get centroid coordinates
            lm_centroid = neuron_group['lm_centroid_bw'][:].round().astype(int)
            z, y, x = lm_centroid

            # Check if centroid is within mask bounds
            if (z < 0 or y < 0 or x < 0 or
                    z >= z_max or y >= y_max or x >= x_max):
                # outside_lm_points.append(lm_centroid)
                neuron_group.attrs['g_coeff'] = np.nan
                neuron_group.attrs['r_coeff'] = np.nan
                continue

            # Get coefficients from masks at centroid position
            try:
                c0_coeff = mask_colored_stack_c0[z, y, x]
                neuron_group.attrs['g_coeff'] = c0_coeff
            except IndexError:
                # outside_lm_points.append(lm_centroid)
                neuron_group.attrs['g_coeff'] = np.nan

            try:
                c1_coeff = mask_colored_stack_c1[z, y, x]
                neuron_group.attrs['r_coeff'] = c1_coeff
            except IndexError:
                # outside_lm_points.append(lm_centroid)
                neuron_group.attrs['r_coeff'] = np.nan

INs = get_neurons_with_attribute(lut_path, "r_coeff", 0.8, ">")
#print(INs)

# Visualizing INs 

In [139]:
r_in_centroids = []
r_out_centroids = []
r_pos_centroids = []
r_neg_centroids = []
c1_threshold = 0.8

with h5py.File(lut_path, 'r') as f:
    for neuron, neuron_group in f['neurons'].items():
        if 'lm_centroid_bw' in neuron_group:
            centroid = neuron_group['lm_centroid_bw'][:]
            
            # Check if coefficients exist
            if 'r_coeff' in neuron_group.attrs:
                r_coeff = neuron_group.attrs['r_coeff']
                
                if np.isnan(r_coeff):
                    r_out_centroids.append(centroid)
                else:
                    r_in_centroids.append(centroid)
                    if r_coeff > c1_threshold:
                        r_pos_centroids.append(centroid)
                    else:
                        r_neg_centroids.append(centroid)

viewer = napari.Viewer()
#viewer.add_image(lm_stack[:, 1], blending="additive", name="lm_stack")
viewer.add_image(mask_colored_stack_c1, blending="additive", name="c1_mask")
viewer.add_points(r_in_centroids, face_color="green", name="r_in_centroids")
viewer.add_points(r_out_centroids, face_color="red", name="r_out_centroids")
viewer.add_points(r_pos_centroids, face_color="cyan", name="r_pos_centroids")
viewer.add_points(r_neg_centroids, face_color="magenta", name="r_neg_centroids")

2025-02-05 16:38:25,367 - WARNING - DirectWrite: CreateFontFaceFromHDC() failed (Indicates an error in an input file such as a font file.) for QFontDef(Family="", pointsize=12, pixelsize=16, styleHint=5, weight=50, stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0, lfHeight=-16) dpi=96
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Points layer 'r_neg_centroids' at 0x1740206f130>

# Visualizing LUT mapping

In [128]:

lm_centroids_stored = []
em_centroids_stored = []
ng_centroids_stored = []
em_centroids_ds_stored = []

with h5py.File(lut_path, 'r+') as f:
    for neuron, neuron_group in f['neurons'].items():
        if 'lm_centroid_bw' in neuron_group:
            lm_centroids_stored.append(list(neuron_group['lm_centroid_bw'][:]))
            em_centroids_stored.append(list(neuron_group['em_centroid_bw'][:]))
            ng_centroids_stored.append(list(neuron_group['em_centroid_ng'][:]))
            em_centroids_ds_stored.append(list(neuron_group['em_centroid_ds'][:]))

# viewer = napari.Viewer()

# Generate a color cycle using hsv colormap which gives better color separation
num_colors = 10000  # Increased number of colors
colors = plt.cm.magma(np.linspace(0, 1, num_colors))

# Add points with the larger color cycle
properties = {
    'point_index': np.arange(len(lm_centroids_stored))
}

viewer.add_points(
    lm_centroids_stored,
    name="lm_centroids_stored",
    properties=properties,
    face_color='point_index',
    face_color_cycle=colors,
    size=10
)

viewer.add_points(
    em_centroids_stored,
    name="em_centroids_stored",
    properties=properties,
    face_color='point_index',
    face_color_cycle=colors,
    size=10
)

viewer.add_points(
    ng_centroids_stored,
    name="ng_centroids_stored",
    properties=properties,
    face_color='point_index',
    face_color_cycle=colors,
    size=100
)

viewer.add_points(
    em_centroids_ds_stored,
    name="em_centroids_ds_stored",
    properties=properties,
    face_color='point_index',
    face_color_cycle=colors,
    size=10
)

<Points layer 'em_centroids_ds_stored' at 0x1def358b070>

In [233]:
# Creating Dataframe from HDF5

# Initialize empty lists to store the data
data = []

with h5py.File(lut_path, 'r') as f:
    for neuron_name, neuron_group in tqdm(f['neurons'].items(), desc="Extracting neuron info"):
        neuron_id = neuron_name.split('_')[1]  # Extract ID from 'neuron_XX'

        # Create dictionary for this neuron
        neuron_data = {
            'neuron_id': neuron_id,
            'em_centroid_bw': neuron_group['em_centroid_bw'][:] if 'em_centroid_bw' in neuron_group else None,
            'em_centroid_ds': neuron_group['em_centroid_ds'][:] if 'em_centroid_ds' in neuron_group else None,
            'em_centroid_ng': neuron_group['em_centroid_ng'][:] if 'em_centroid_ng' in neuron_group else None,
            'lm_centroid_bw': neuron_group['lm_centroid_bw'][:] if 'lm_centroid_bw' in neuron_group else None,
            'lm_centroid_plane_bw': neuron_group['lm_centroid_plane_bw'][
                                    :] if 'lm_centroid_plane_bw' in neuron_group else None,
            'lm_centroid_plane_flip': neuron_group['lm_centroid_plane_flip'][
                                      :] if 'lm_centroid_plane_flip' in neuron_group else None,
            'agglo_id': neuron_group.attrs.get('agglo_id', None),
            'g_coeff': neuron_group.attrs.get('g_coeff', None),
            'in_OB': neuron_group.attrs.get('in_OB', None),
            'lm_stack_bw_mask_label': neuron_group.attrs.get('lm_stack_bw_mask_label', None),
            'r_coeff': neuron_group.attrs.get('r_coeff', None)
        }
        data.append(neuron_data)

# Create DataFrame
df = pd.DataFrame(data)

df.head(10)

Extracting neuron info: 100%|██████████| 9462/9462 [00:44<00:00, 214.07it/s]


,neuron_id,em_centroid_bw,em_centroid_ds,em_centroid_ng,lm_centroid_bw,lm_centroid_plane_bw,lm_centroid_plane_flip,agglo_id,g_coeff,in_OB,lm_stack_bw_mask_label,r_coeff
0,0,"[585.8534150215567, 283.09280689811663, 411.51...","[4.0, 411.0, 410.0]","[7731.3453596550935, 9385.172452915816, 79.515...","[116.81865840534263, 348.35750448005194, 282.4...","[41.1808084289358, 71.6877313292895, 173.19601...","[7.0, 220.34782608695653, 466.82608695652175]",16922938,0.324468,True,3425,0.000000
1,1,"[564.9634206623826, 281.9189322787939, 392.601...","[6.0, 392.0, 431.0]","[8065.585269401878, 9082.630581644422, 98.2970...","[108.89449801384788, 347.8131157787848, 269.19...",None,None,16105976,0.252260,True,3197,0.000000
2,10,"[542.0241465445462, 282.49042464612825, 438.31...","[5.0, 438.0, 454.0]","[8432.613655287261, 9814.053566472383, 89.1532...","[91.33075592841419, 340.9222018484342, 288.511...",None,None,17771455,0.382576,True,2707,0.020202
3,100,"[585.015600809813, 266.69917827795643, 385.776...","[21.0, 385.0, 411.0]","[7744.750387042992, 8973.427295462665, 341.813...","[123.35946898945797, 335.23803581131574, 262.6...",None,None,16090441,0.010780,True,3679,0.000000
4,1000,"[486.2682801235839, 213.55887401304497, 264.08...","[74.0, 264.0, 509.0]","[9324.707518022657, 7026.320974939925, 1192.05...","[103.19270962806391, 287.6855738424987, 172.55...","[42.498683512068645, 190.2219401622382, 9.8804...","[6.0, 54.4263959390863, 32.243654822335024]",107744409,0.222127,True,2996,0.043739
5,1001,"[458.74672144098594, 214.0944857007426, 275.66...","[73.0, 275.0, 537.0]","[9765.052456944224, 7211.572286301153, 1183.48...","[87.68328487503518, 283.6688359322946, 174.250...",None,None,11190379,0.264107,True,2537,0.072884
6,1002,"[372.83935135135135, 214.43394594594594, 291.3...","[73.0, 291.0, 623.0]","[11139.57037837838, 7463.389621621622, 1178.05...","[45.31529076586182, 273.76545658072115, 172.07...",None,None,61171657,0.177879,True,1223,0.343216
7,1003,"[576.4505971769815, 211.95284628509384, 296.92...","[76.0, 296.0, 419.0]","[7881.790445168295, 7551.781448735846, 1217.75...","[140.9114035706243, 290.40611983088957, 198.59...",None,None,60212755,0.029945,True,4315,0.000000
8,1004,"[394.8062542488103, 212.82936777702244, 284.55...","[75.0, 284.0, 601.0]","[10788.099932019035, 7353.821210061183, 1203.7...","[55.7237030916078, 275.74639916217484, 171.049...",None,None,57811036,0.412852,True,1600,0.905810
9,1005,"[446.06441393875394, 213.63287574797607, 333.9...","[74.0, 334.0, 549.0]","[9967.969376979938, 8144.242520239352, 1190.87...","[74.07559995774915, 274.6973086178733, 200.357...",None,None,109457145,0.066140,True,2218,0.000987


In [238]:
# How many cells are recorded in OB?
print(f"From {len(df)} neurons:")
# How many cells are interneurons (IN)?
IN_th = 0.5
all_INs = df[df["r_coeff"] > IN_th]
print(f"There are {len(all_INs)} INs, with an intensity of {IN_th * 100}%. ")

neurons_in_ob = df[df["in_OB"]]
neurons_in_ob.head(10)
print(f"There are {len(neurons_in_ob)} neurons in OB")

# How many cells in OB are interneurons (IN)?
INs_in_ob = neurons_in_ob[neurons_in_ob["r_coeff"] > IN_th]
print(f"There are {len(INs_in_ob)} INs, with an intensity of {IN_th * 100}%. ")

neurons_in_ob_with_activity = neurons_in_ob[neurons_in_ob["lm_centroid_plane_flip"].notna()]
print(f"There are {len(neurons_in_ob_with_activity)} neurons in OB with activity")

INs_in_ob_with_activity = neurons_in_ob_with_activity[neurons_in_ob_with_activity["r_coeff"] > IN_th]
print(f"There are {len(INs_in_ob_with_activity)} INs, with an intensity of {IN_th * 100}%. ")
neurons_in_ob_with_activity.head()

From 9462 neurons:
There are 888 INs, with an intensity of 50.0%. 
There are 2570 neurons in OB
There are 150 INs, with an intensity of 50.0%. 
There are 389 neurons in OB with activity
There are 22 INs, with an intensity of 50.0%. 


,neuron_id,em_centroid_bw,em_centroid_ds,em_centroid_ng,lm_centroid_bw,lm_centroid_plane_bw,lm_centroid_plane_flip,agglo_id,g_coeff,in_OB,lm_stack_bw_mask_label,r_coeff
0,0,"[585.8534150215567, 283.09280689811663, 411.51...","[4.0, 411.0, 410.0]","[7731.3453596550935, 9385.172452915816, 79.515...","[116.81865840534263, 348.35750448005194, 282.4...","[41.1808084289358, 71.6877313292895, 173.19601...","[7.0, 220.34782608695653, 466.82608695652175]",16922938,0.324468,True,3425,0.000000
4,1000,"[486.2682801235839, 213.55887401304497, 264.08...","[74.0, 264.0, 509.0]","[9324.707518022657, 7026.320974939925, 1192.05...","[103.19270962806391, 287.6855738424987, 172.55...","[42.498683512068645, 190.2219401622382, 9.8804...","[6.0, 54.4263959390863, 32.243654822335024]",107744409,0.222127,True,2996,0.043739
28,1022,"[501.1045652173913, 214.37326086956523, 774.44...","[73.0, 774.0, 494.0]","[9087.326956521738, 15192.182608695652, 1179.0...","[47.66943511067517, 221.93346664281353, 425.23...","[40.95637162987732, 153.13990328233683, 336.26...","[6.0, 20.839080459770116, 336.91379310344826]",122735082,0.000000,True,0,0.000000
34,1028,"[265.43612818261636, 211.79894644424934, 353.8...","[76.0, 353.0, 730.0]","[12858.021949078138, 8463.374012291484, 1220.2...","[-8.746478272813448, 246.75313971846313, 187.3...","[34.60160687746654, 212.52634554540236, 243.05...","[6.0, 76.40287769784173, 249.8273381294964]",62870187,NaN,True,False,NaN
49,1041,"[362.3381984036488, 212.56602052451538, 536.34...","[75.0, 536.0, 633.0]","[11307.58882554162, 11382.458608893956, 1207.9...","[14.498656875303254, 239.43332657832468, 286.6...","[42.49244980754802, 167.22001556050478, 166.07...","[6.0, 33.401162790697676, 178.0581395348837]",66892508,0.212851,True,383,0.000000


In [236]:
print(list(neurons_in_ob_with_activity["agglo_id"]))

[16922938, 107744409, 122735082, 62870187, 66892508, 21135025, 120221209, 61094802, 11062648, 64504042, 70255682, 105201237, 55280050, 103488253, 60230501, 68589252, 102686755, 57747936, 110195623, 111892685, 116120335, 21952875, 120143925, 101932129, 103645255, 55296874, 60276706, 109521985, 18605985, 10325655, 12775607, 66939846, 71920220, 110195645, 112805275, 113653459, 17724727, 56159981, 66106749, 21103877, 101053458, 55248881, 108547193, 111091845, 112725661, 115129692, 21936631, 69499523, 103502050, 12822921, 19374672, 112773205, 112725772, 15273470, 56977705, 110227454, 72767896, 103628403, 9492998, 55233695, 107697504, 104383085, 17756311, 104335307, 163588797, 115962443, 69422774, 104445665, 17851656, 119278696, 58674780, 157822322, 72625964, 148566865, 106990946, 107006443, 56914542, 156047206, 69436096, 121053492, 148567797, 11126626, 161891594, 162629666, 20256379, 21905599, 56977781, 105201309, 60386843, 22723108, 73523250, 58594308, 157822716, 15353008, 165049629, 12022

In [237]:
print(list(INs_in_ob_with_activity["agglo_id"]))

[70255682, 10325655, 21103877, 12022377, 156847689, 162661378, 155999482, 150186863, 202635221, 59492268, 155229794, 197623949, 205982004, 198440199, 212613531, 62021152, 107903622, 250119849, 58674716, 250119847, 212645111, 197702898]
